# Тестовое задание

## Цель
Вам необходимо обработать сырые данные, поступившие от парсера.

## Описание
В файле файл с данными две вкладки `platform1`, `platform2`. Это данные о гостиницах от двух разных платформ в одном регионе.

## Поля

 - `id` - Идентификатор в БД
 - `create_time` - Дата создания
 - `title` - Название
 - `hotel_type_original` - Тип гостиницы
 - `city` - Город
 - `address` - Адрес
 - `rating` - Рейтинг
 - `rating_5` - Рейтинг по 5-ти бальной шкале
 - `review_count` - Количество отзывов
 - `star_rating` - Звездность
 - `rooms_count` - Количество номеров
 - `contact_social` - Контакты соц. сетей
 - `description` - Описание
 - `email` - email строкой, несколько значений через запятую
 - `phone` - телефон строкой, несколько значений через запятую
 - `website` - сайты строкой, несколько значений через запятую
 - `parsing_time` - Время сбора
 - `lat` - Широта
 - `lon` - Долгота
 - `uid` - Уникальный идентификатор гостиницы на платформе, не может быть разным у одной гостиницы, и не может повторятся у разных гостиниц, но в рамках одной платформы
 
## Состав данных

 - Записи могут повторятся, и самые актуальные данные идут в конце.
 - Если какое-то поле по конкретной гостинице пустое в последней записи, но встречалось ранее, то необходимо его взять из более ранних записей.
 - Поля телефона, email, сайтов могу содержать лишние символы, несколько записей и прочее.
 - Формат записи названия, адреса, типа гостиницы отличается в разных платформах.

## Задача

1. Собрать от каждой платформы финальный список гостиниц, в котором по каждой гостинице внутри платформы будет только одна запись с самыми актуальными и полными данными.
2. Почистить данные.
3. Поля телефона, email, сайтов распарсить и сохранить как списки в одинаковом формате (address@domen.org, 79234553322, domen.ru)
4. Вывести топ 10 по каждой платформе, по параметрам: 
   - a. больше всего телефонов,
   - b. больше всего отзывов.
5. Вывести квадрат координат размером 1км на 1км, где больше всего гостиниц.
6. Задача со *, объединить данные от двух платформ, по критерию, который вы придумаете
   - a. Вывести все гостиницы, которые есть в платформе 1 и нет в платформе 2
   - b. Вывести топ 10 гостиниц которые есть в обоих платформах, по суммарному количеству отзывов

## Требования

1. Результат должен быть представлен Jupyter notebook
2. Результат должен воспроизводится автоматически и нуля при повторном запуске, ручные правки должны быть учтены в коде
3. Комментарии приветствуются

### Шаг 1. Импорт данных

**1. Загрузка библиотек**

In [1]:
import pandas as pd
import re

**2. Загрузка первого, второго датасета и вывод первые их строк**

In [2]:
platform1 = pd.read_excel('C:\\Users\\Nemo\Downloads\\hotels_parsing_result.xlsx', sheet_name='platform1')
platform2 = pd.read_excel('C:\\Users\\Nemo\\Downloads\\hotels_parsing_result.xlsx', sheet_name='platfrom2')

In [3]:
display(platform1.head())

,id,create_time,title,hotel_type_original,city,address,rating,rating_5,review_count,region,...,rooms_count,contact_social,description,email,phone,website,uid,parsing_time,lat,lon
0,81842,2023-09-26 14:45:42.910962,Апартаменты Baltia Западный Пляж,Apartment,Зеленоградск,"улица Приморская, д. 31, Зеленоградск",10.0,5.00,0,2,...,NaN,NaN,NaN,NaN,NaN,NaN,10595357,2023-09-26 20:56:02.013106,54.951103,20.456526
1,86341,2023-09-26 15:01:55.184815,Отель Авиатор,Hotel,Уфа,"улица Мушникова, д.28, Уфа",8.7,4.35,5,2,...,8.0,NaN,NaN,NaN,NaN,NaN,8332081,2023-09-26 20:56:02.013106,54.783913,56.116150
2,86420,2023-09-26 15:02:12.005085,Клеопатра,Hotel,Уфа,"Коммунистическая ул., д. 53, Уфа",6.3,3.15,10,2,...,15.0,NaN,NaN,NaN,NaN,NaN,7724907,2023-09-26 20:56:02.013106,54.726463,55.946445
3,86421,2023-09-26 15:02:12.225334,Татьяна,Hotel,Сибай,"Учалинская улица, 37, Сибай",10.0,5.00,2,2,...,15.0,NaN,NaN,NaN,NaN,NaN,6598197,2023-09-26 20:56:02.013106,52.719230,58.658596
4,85898,2023-09-26 15:00:19.217359,Мини-Отель Чемодан,Mini-hotel,Стерлитамак,"улица Дружбы, д.28 Д, Стерлитамак",8.0,4.00,0,2,...,23.0,NaN,NaN,NaN,NaN,NaN,8622634,2023-09-26 20:56:02.013106,53.642567,55.932686


In [4]:
display(platform2.head())

,id,create_time,title,hotel_type_original,city,address,rating,rating_5,review_count,region,...,rooms_count,contact_social,description,email,phone,website,uid,parsing_time,lat,lon
0,4608,2023-09-14 15:46:07.855977,"Уют, гостиница",hotel,NaN,"Россия, Республика Башкортостан, Аскинский рай...",NaN,NaN,NaN,2,...,5.0,NaN,NaN,NaN,NaN,NaN,70000001064499291,2023-09-14 22:44:57.013238,NaN,NaN
1,4815,2023-09-14 15:46:48.332492,Гостиница Ашкадар,NaN,sterlitamak,"Россия, Республика Башкортостан, Стерлитамак г...",NaN,NaN,NaN,2,...,NaN,NaN,NaN,NaN,NaN,NaN,7600610350202940,2023-09-14 22:44:57.054250,NaN,NaN
2,4750,2023-09-14 15:46:35.659931,"Островок, база отдыха",hotel_rest,NaN,"Россия, Республика Башкортостан, Гафурийский р...",NaN,NaN,NaN,2,...,NaN,NaN,NaN,NaN,NaN,NaN,70000001076845858,2023-09-14 22:44:57.043234,NaN,NaN
3,4786,2023-09-14 15:46:42.672665,"Ардан, база отдыха",hotel_rest,NaN,"Россия, Республика Башкортостан, Баймакский ра...",NaN,NaN,NaN,2,...,NaN,NaN,NaN,NaN,NaN,NaN,70000001075946655,2023-09-14 22:44:57.049244,NaN,NaN
4,16310,2023-09-15 11:56:48.785266,Гостиница,hotel,beloretsk,"Россия, Республика Башкортостан, Белорецкий ра...",NaN,NaN,NaN,2,...,NaN,NaN,NaN,NaN,NaN,NaN,70000001031030911,2023-09-15 18:55:23.191778,53.969912,58.400557


Импортировали библиотеку, модуль, и вывели первые строки датафреймов

### 2. Обработка дубликатов и объединение данных

Чтобы собрать финальный список гостиниц для каждой платформы:

 - Нужно убедится, что у нас есть уникальные записи для каждой гостиницы.
 - Нужно использовать функцию `groupby` и `last` для сохранения самых актуальных данных.

In [5]:
# Группировка данных по уникальному идентификатору (uid) и сохранение последней записи
platform1_final = platform1.groupby('uid').last().reset_index()
platform2_final = platform2.groupby('uid').last().reset_index()

Сгруппировали данных по уникальному идентификатору (uid), сохранив последние записи.

### 3. Заполнение пропущенных данных

Необходимо заполнить пропуски данными из более ранних записей. Используя метод `fillna`.

In [6]:
# Заполняем пропуски на основе более ранних записей
platform1_final = platform1.groupby('uid', group_keys=False).apply(lambda x: x.ffill().bfill()).reset_index(drop=True)
platform2_final = platform2.groupby('uid', group_keys=False).apply(lambda x: x.ffill().bfill()).reset_index(drop=True)

Заполнили пропуски данными из более ранних версий

### 4. Чистка данных и нормализация полей

Для полей с телефонами, email и сайтами нужно удалить лишние символы и разбить строки на списки.

In [7]:
def clean_contact_info(data):
    # Очистка телефонов
    data['phone'] = data['phone'].apply(lambda x: re.findall(r'\d+', str(x)))
    # Очистка email
    data['email'] = data['email'].apply(lambda x: re.findall(r'\S+@\S+', str(x)))
    # Очистка сайтов
    data['website'] = data['website'].apply(lambda x: re.findall(r'[\w.-]+(?:\.[\w\.-]+)+', str(x)))
    return data

platform1_final = clean_contact_info(platform1_final)
platform2_final = clean_contact_info(platform2_final)

Сохранили адреса сайтов, электронных почт и номеров телефонов в одинаковом формате (address@domen.org, 79234553322, domen.ru)

### 5. Вывод топ-10 гостиниц по количеству телефонов и отзывов

In [8]:
# Подсчет количества телефонов для каждой гостиницы
platform1_final['phone_count'] = platform1_final['phone'].apply(len)
platform2_final['phone_count'] = platform2_final['phone'].apply(len)

# Топ-10 по количеству телефонов
platform1_top_phones = platform1_final.nlargest(10, 'phone_count')
platform2_top_phones = platform2_final.nlargest(10, 'phone_count')

# Удаление дубликатов на основе названия и количества отзывов
platform1_top_phones_unique = platform1_top_phones.drop_duplicates(subset=['title', 'phone_count'])
platform2_top_phones_unique = platform2_top_phones.drop_duplicates(subset=['title', 'phone_count'])

# Отображение уникальных результатов
display(platform1_top_phones_unique[['title', 'phone_count']])
display(platform2_top_phones_unique[['title', 'phone_count']])

,title,phone_count
0,Апартаменты Baltia Западный Пляж,0
1,Отель Авиатор,0
2,Клеопатра,0
3,Татьяна,0
4,Мини-Отель Чемодан,0
5,Hilton Garden Inn Ufa Riverside,0
6,Отель Hampton by Hilton Уфа,0
7,Отель ВолнаСити,0
8,AZIMUT Сити Отель Уфа,0
9,Отель Ural Taur,0


,title,phone_count
2653,"Панорама, мини-отель",6
2671,Goldy Land,6
2718,"Ивкор, база отдыха",5
2732,"Берлога на озере Калкан, база отдыха",5
383,"Vgosti, гостиница",4
2221,"Аэропорт aeropot hotel, гостиница",4
2274,"Добр здравич, гостиница",4


In [9]:
# Топ-10 по количеству отзывов
platform1_top_reviews = platform1_final.nlargest(10, 'review_count')
platform2_top_reviews = platform2_final.nlargest(10, 'review_count')

# Удаление дубликатов на основе названия и количества отзывов
platform1_top_reviews_unique = platform1_top_reviews.drop_duplicates(subset=['title', 'review_count'])
platform2_top_reviews_unique = platform2_top_reviews.drop_duplicates(subset=['title', 'review_count'])

# Отображение уникальных результатов
display(platform1_top_reviews_unique[['title', 'review_count']])
display(platform2_top_reviews_unique[['title', 'review_count']])

,title,review_count
1741,Radisson Blu Hotel Kaliningrad,1201
8,AZIMUT Сити Отель Уфа,988
1826,Отель Кайзерхоф (Kaiserhof),930
5485,Отель Ibis Калининград Центр,837
5,Hilton Garden Inn Ufa Riverside,616
11,Отель АМАКС Сити-отель,540
6,Отель Hampton by Hilton Уфа,460
193,Гостиничный Комплекс Президент Отель,414
5482,Гостиница Мартон Палас Калининград,334
1578,"Гостиница ""Шкиперская""",302


,title,review_count
17,"Hilton Garden Inn Ufa Riverside, отель",823.0
50,"Башкирия, гостиничный комплекс",780.0


 - В первом датафреме телефонов не было. Во втором датафрейме удалось собрать только 7 отелей, где были указаны номера телефонов.
 - По первому датафрейму вывели топ-10 отелей по количеству отзывов. По второму датафрейму удалось вывести только 2 отеля.

### 6. Поиск квадрата с максимальным числом гостиниц

Для нахождения квадрата координат 1 км на 1 км с максимальным числом гостиниц, нужно округлить координаты с точностью до нужного уровня.

In [10]:
# Округление координат до 3-х знаков после запятой (~1 км)
platform1_final['lat_rounded'] = platform1_final['lat'].round(3)
platform1_final['lon_rounded'] = platform1_final['lon'].round(3)

platform2_final['lat_rounded'] = platform2_final['lat'].round(3)
platform2_final['lon_rounded'] = platform2_final['lon'].round(3)

# Группировка по округленным координатам и подсчет количества гостиниц
platform1_hotspot = platform1_final.groupby(['lat_rounded', 'lon_rounded']).size().reset_index(name='count')
platform2_hotspot = platform2_final.groupby(['lat_rounded', 'lon_rounded']).size().reset_index(name='count')

# Найденные максимальные кластеры гостиниц
display(platform1_hotspot.nlargest(1, 'count'))
display(platform2_hotspot.nlargest(1, 'count'))

,lat_rounded,lon_rounded,count
1225,54.724,55.943,44


,lat_rounded,lon_rounded,count
380,54.761,56.391,16


По каждому из датафреймов, нашли квадрат с максимальным числом гостиниц.

### 7. Задача со звездочкой: объединение данных
Для объединения данных от двух платформ по критерию (например, по названию или UID):

In [11]:
# Объединение по названию
merged_data = pd.merge(platform1_final, platform2_final, on='title', suffixes=('_plat1', '_plat2'))

# Гостиницы, которые есть в платформе 1, но нет в платформе 2
only_in_platform1 = platform1_final[~platform1_final['title'].isin(platform2_final['title'])]
only_in_platform1_unique = only_in_platform1.drop_duplicates(subset=['title', 'city']) # Удаление дубликатов

# Топ-10 гостиниц по суммарному количеству отзывов
merged_data['total_reviews'] = merged_data['review_count_plat1'] + merged_data['review_count_plat2']
top_10_reviews = merged_data.nlargest(10, 'total_reviews')
top_10_reviews_unique = top_10_reviews.drop_duplicates(subset=['title', 'total_reviews']) # Удаление дубликатов

# Отображение уникальных значений
display(only_in_platform1_unique[['title', 'city']])
display(top_10_reviews_unique[['title', 'total_reviews']])

,title,city
0,Апартаменты Baltia Западный Пляж,Зеленоградск
1,Отель Авиатор,Уфа
2,Клеопатра,Уфа
3,Татьяна,Сибай
4,Мини-Отель Чемодан,Стерлитамак
...,...,...
5517,Podushka Hostel,Уфа
5518,Lux apartament UFA,Уфа
5519,Биатлон,Уфа
5520,Лидо,Уфа


,title,total_reviews
0,AZIMUT Сити Отель Уфа,1374.0
15,Villa Blanca,62.0


 - Объеденили данных из 2-х датафреймов по названию.
 - Вывели гостинницы, которые есть в платформе 1, но нет в платформе 2.
 - Вывели гостинницы, по суммарному количеству отзывов.